In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import requests
import time
from collections import defaultdict
from selenium.webdriver.common.by import By

In [3]:
##Options for chromedriver, when using selenium, does not matter when using Collab since it acts more like a remote machine
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [5]:
cricbuzz_highlights_url='https://www.cricbuzz.com/cricket-full-commentary/45901/gt-vs-lsg-4th-match-indian-premier-league-2022'

In [ ]:
#https://www.cricbuzz.com/cricket-series/5945/indian-premier-league-2023

In [6]:
driver.get(cricbuzz_highlights_url)

In [7]:
cricbuzz_soup= BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
div_tag=cricbuzz_soup.find_all('div',{'class':"cb-col cb-col-100 ng-scope"})

In [9]:
#class="cb-col cb-col-100 ng-scope"
#class="cb-col cb-col-100 ng-scope"
#class="cb-col cb-col-100 ng-scope"
#class="cb-com-ln ng-binding ng-scope cb-col cb-col-90"
#class="cb-col cb-col-20" match box
#class=cb-nav-subhdr cb-font-12  header 
#class="cb-billing-plans-text cb-team-lft-item" team names

In [10]:
match_metadata_values=[c.next_element.next_element.next_element.text.replace('\xa0','') for c in cricbuzz_soup.find_all('span',{'class':'text-bold'})]

match_metadata_values



['Indian Premier League 2022', 'Wankhede Stadium,Mumbai', 'Mar 28,']

In [11]:
year=match_metadata_values[0].split()[-1]
series="IPL"
venue=match_metadata_values[1]
date=match_metadata_values[2]+year

In [12]:
def match_no():
    team_names=cricbuzz_soup.find('div',{'class':"cb-billing-plans-text cb-team-lft-item"})
    teams=team_names.text.split(',')[1]
    return teams.strip()[0:3]

In [13]:
#cb-com-ln ng-binding ng-scope cb-col cb-col-90
def find_toss():
    toss=cricbuzz_soup.find_all('p',{'class':"cb-com-ln ng-binding ng-scope cb-col cb-col-90"})
    for i in toss:
        if "won the toss" in i.get_text().lower():
            return i.get_text()


In [14]:
def team_names():
    team_names=cricbuzz_soup.find('div',{'class':"cb-billing-plans-text cb-team-lft-item"})
    teams=team_names.text.split(',')[0]
    teams=teams.split('vs')
    teams=[team.strip() for team in teams]
    return teams

In [15]:
def match_info():
    match_details=cricbuzz_soup.find('div',{'class':"cb-col cb-col-20"})
    match_info=""
    for i in match_details:
        match_info+=i.get_text()
    return match_info    


In [16]:
def find_team_subs(team):
    preview_obj=cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})
    for tag in preview_obj:
        if tag.b==None:
            pass
        else:
            if "subs" in tag.get_text().lower():
                print(tag.get_text())
                return tag.get_text().split(',')[0:]

In [17]:
def find_team_playing11(team):
    preview_obj=cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})
    for tag in preview_obj:
        if tag.b==None:
            pass
        else:
            if team+" (Playing XI)" in tag.get_text():
                return tag.get_text().split(',')[0:]
    

In [18]:
df=pd.DataFrame(columns=['match_no','series','season','team_a','team_b','team_a_11','team_b_11','team_a_subs','team_b_subs','venue','date','toss'])

In [19]:
match =match_no()
df.at[1,'match_no']=match

In [20]:
team_a,team_b=team_names()

In [21]:
df.at[1,'team_a']=team_a
df.at[1,'team_b']=team_b

In [22]:
df.at[1,'team_a_11']=find_team_playing11(team_a)
df.at[1,'team_b_11']=find_team_playing11(team_b)
df.at[1,'team_a_subs']=find_team_subs(team_a)
df.at[1,'team_b_subs']=find_team_subs(team_b)
df.at[1,'series']=series
df.at[1,'season']=year
df.at[1,'venue']=venue
df.at[1,'date']=date
df.at[1,'toss']=find_toss()

In [23]:
display(df)

,match_no,series,season,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,venue,date,toss
1,4th,IPL,2022,Gujarat Titans,Lucknow Super Giants,"[Gujarat Titans (Playing XI): Shubman Gill, M...",[Lucknow Super Giants (Playing XI): KL Rahul(c...,None,None,"Wankhede Stadium,Mumbai","Mar 28,2022",Gujarat Titans have won the toss and have opte...


In [24]:
df.loc[1,'team_a_11']

['Gujarat Titans (Playing XI): Shubman Gill',
 ' Matthew Wade(w)',
 ' Vijay Shankar',
 ' Abhinav Manohar',
 ' Hardik Pandya(c)',
 ' David Miller',
 ' Rahul Tewatia',
 ' Rashid Khan',
 ' Lockie Ferguson',
 ' Varun Aaron',
 ' Mohammed Shami']

In [25]:
link_text=[]
div_tag=cricbuzz_soup.find_all('div',{"class":"cb-hig-pil ng-scope"})
for a_tag in div_tag:
    link_text.append(a_tag.a.string)
link_text=link_text[1:]
link_text

['LSG Inns', 'GT Inns']

In [26]:
cricbuzz_page_soup=[]
##Iterate through the link textss
for l in link_text:
        ##Click the innings text
        print(l,' Analysed')
        loadMoreButton=driver.find_element(By.LINK_TEXT,l)
        loadMoreButton.click()
        ##Click the 'dropped catches' text box, or the option chosen above. 
        #l2=driver.find_element(By.LINK_TEXT,option)
        #l2.click()
        time.sleep(5)
        ##Collect the soup of the existing view that has been clicked by the driver element
        cricbuzz_soup_inner=BeautifulSoup(driver.page_source, 'html.parser')

        ##Append each of the page contents to a list
        cricbuzz_page_soup.append(cricbuzz_soup_inner)

##Scraping has been completed
print ("Full scraping of key events complete...")
##Stop the driver element
driver.quit()

LSG Inns  Analysed
GT Inns  Analysed
Full scraping of key events complete...


In [31]:
match_commentary_df=pd.DataFrame()
for inn_num,cricbuzz_highlights_soup in enumerate(cricbuzz_page_soup):
    cricbuzz_innings_soup=cricbuzz_highlights_soup.find_all('div',{'class':'cb-col cb-col-8 text-bold ng-scope'})
    ball_overs=[]
    ball_commentary=[]
    for cinn_soup in cricbuzz_innings_soup:
        ball_overs.append(cinn_soup.text)
        ball_commentary.append(str(cinn_soup.find_next('p').text))
        #ball_commentry.append()
    innings_commentary_df=pd.DataFrame({'ball':ball_overs,'Commentary Text':ball_commentary})
    innings_commentary_df['ball']=innings_commentary_df['ball']
    innings_commentary_df['innings']=link_text[inn_num]
    match_commentary_df=pd.concat([match_commentary_df,innings_commentary_df])

In [32]:
match_commentary_df.reset_index(inplace=True,drop=True)
match_commentary_df

,ball,Commentary Text,innings
0,19.6,"Aaron to Krunal Pandya, 1 run, hard length an...",LSG Inns
1,19.5,"Aaron to Chameera, 1 run, banged into the sur...",LSG Inns
2,19.4,"Aaron to Ayush Badoni, out Caught by Hardik P...",LSG Inns
3,19.3,"Aaron to Krunal Pandya, 1 run, slower short o...",LSG Inns
4,19.2,"Aaron to Krunal Pandya, FOUR, not sure what h...",LSG Inns
...,...,...,...
240,0.4,"Chameera to Shankar, no run, 140ks good lengt...",GT Inns
241,0.4,"Chameera to Shankar, wide, short and down the...",GT Inns
242,0.3,"Chameera to Shubman Gill, out Caught by Hooda...",GT Inns
243,0.2,"Chameera to Shubman Gill, no run, 137ks on a ...",GT Inns


In [33]:
match_commentary_df

,ball,Commentary Text,innings
0,19.6,"Aaron to Krunal Pandya, 1 run, hard length an...",LSG Inns
1,19.5,"Aaron to Chameera, 1 run, banged into the sur...",LSG Inns
2,19.4,"Aaron to Ayush Badoni, out Caught by Hardik P...",LSG Inns
3,19.3,"Aaron to Krunal Pandya, 1 run, slower short o...",LSG Inns
4,19.2,"Aaron to Krunal Pandya, FOUR, not sure what h...",LSG Inns
...,...,...,...
240,0.4,"Chameera to Shankar, no run, 140ks good lengt...",GT Inns
241,0.4,"Chameera to Shankar, wide, short and down the...",GT Inns
242,0.3,"Chameera to Shubman Gill, out Caught by Hooda...",GT Inns
243,0.2,"Chameera to Shubman Gill, no run, 137ks on a ...",GT Inns
